In [131]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

books = pd.read_csv('BX-Books.csv', sep=';', on_bad_lines='skip', encoding='latin-1')


In [ ]:
books.head()

In [ ]:
books.columns

In [ ]:
books=books[['ISBN', 'Book-Title','Book-Author', 'Year-Of-Publication', 'Publisher' ]]

In [ ]:
books.head()

In [ ]:
books.rename(columns={'Book-Title':'Title','Book-Author': 'Author' ,'Year-Of-Publication':'Year' })

In [ ]:
users=pd.read_csv('BX-Users.csv',  sep=';', on_bad_lines='skip', encoding='latin-1')

In [ ]:
users.head()

In [ ]:
users.rename(columns={'User-ID':'User'	}, inplace=True)

In [ ]:
users.head()

In [ ]:
Rating=pd.read_csv('BX-Book-Ratings.csv',sep=';', on_bad_lines='skip', encoding='latin-1')

In [ ]:
Rating.head()

In [ ]:
Rating.rename(columns={'User-ID' : 'User','Book-Rating':'Rating'}, inplace=True)

In [ ]:
Rating.head(2)

In [ ]:
books.shape

In [ ]:
users.shape

In [ ]:
Rating.shape

In [ ]:
x=Rating['User'].value_counts()>200

In [ ]:
x[x].shape

In [ ]:
y=x[x].index

In [ ]:
y

In [ ]:
Rating=Rating[Rating['User'].isin(y)]

In [ ]:
Rating.shape

In [ ]:
Rating.head()

In [ ]:
Rating_with_books=Rating.merge(books,on='ISBN')

In [ ]:
Rating_with_books.shape

In [ ]:
number_rating=Rating_with_books.groupby('Book-Title')['Rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'Rating':'number of ratings'},inplace=True)

In [ ]:
number_rating

In [ ]:
final_rating=Rating_with_books.merge(number_rating, on='Book-Title')

In [ ]:
final_rating

In [ ]:
final_rating.shape

In [ ]:
final_rating=final_rating[final_rating['number of ratings']>=50]

In [ ]:
final_rating.shape

In [ ]:
#print(final_rating.columns)



In [ ]:
#final_rating.columns = final_rating.columns.str.strip()


In [ ]:
final_rating.drop_duplicates(['User', 'Book-Title'], inplace=True)


In [ ]:
final_rating.shape

In [ ]:
book_pivot = final_rating.pivot_table(columns='User', index='Book-Title', values='Rating')


In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0)

In [ ]:
book_pivot.fillna(0, inplace=True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix

# Ensure book_pivot does not have any missing values if necessary
book_pivot.fillna(0, inplace=True)

# Convert the DataFrame to a sparse matrix
book_sparse = csr_matrix(book_pivot)

# To verify, print the sparse matrix
print(book_sparse)


In [ ]:
type(book_sparse)

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:

model=NearestNeighbors(algorithm='brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
book_pivot.index==237

In [ ]:
book_pivot

In [ ]:
distances,suggestions=model.kneighbors(book_pivot.iloc[393,:].values.reshape(1,-1), n_neighbors=6)

In [ ]:
book_pivot

In [ ]:
distances

In [ ]:
suggestions

In [ ]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

In [ ]:
book_pivot.index[339]

In [ ]:
np.where(book_pivot.index=='Animal Farm')[0][0]

In [ ]:
def recommend_book(book_name):
    # Find the index of the book in the pivot table
    book_id = np.where(book_pivot.index == book_name)[0][0]
    
    # Get distances and suggestions (indices of nearest neighbors)
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)
    
    # Print suggestions
    print('The suggestions for', book_name, 'are:')
    
    for i in range(1, len(suggestions[0])):  # Start from 1 to avoid recommending the book itself
        print(f"{i}. {book_pivot.index[suggestions[0][i]]} (Distance: {distances[0][i]:.2f})")


recommend_book('Animal farm')

In [ ]:
recommend_book('Animal Farm')